In [ ]:
# pip install xmltodict
# pip install dart-fss
# pip install opendartreader
# pip install -U finance-datareader
# pip install plotly

In [26]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import xml.etree.ElementTree as et
from io import BytesIO
from zipfile import ZipFile
from bs4 import BeautifulSoup as bs
import xmltodict
import json
import dart_fss as dart
from lxml import html
from xml.etree.ElementTree import parse
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import numpy as np
import datetime
try:
    from pandas import json_normalize
except ImportError:
    from pandas.io.json import json_normalize
import OpenDartReader
from bs4 import BeautifulSoup
import FinanceDataReader as fdr
%matplotlib inline
import matplotlib.pyplot as plt

In [27]:
company_info = pd.read_excel("kosdaq_report_2023.xlsx")
# company_info = pd.read_excel("kosdaq_report_2022.xlsx")

# 엑셀 내용 확인
len(company_info.columns), company_info.columns

(30,
 Index(['고유번호', '종목코드', '회사명', '업종', '주요제품', '상장일', '결산월', '대표자명', '홈페이지', '지역',
        '사업자등록번호', '주소', '업종코드', '설립일', '최대주주_성명', '최대주주_관계', '최대주주_주식종류',
        '최대주주_기초주식수', '최대주주_기초주식지분율', '최대주주_기말주식수', '최대주주_기말주식지분율', '소액주주수',
        '주주수', '소액주주비율', '소액주주_주식수', '총주식수', '소액주주_주식지분율', '감사의견', '감사의견_강조사항',
        '감사의견_핵심감사사항'],
       dtype='object'))

In [28]:
# 결측치 확인
company_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1718 entries, 0 to 1717
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   고유번호          1718 non-null   int64  
 1   종목코드          1718 non-null   int64  
 2   회사명           1718 non-null   object 
 3   업종            1718 non-null   object 
 4   주요제품          1715 non-null   object 
 5   상장일           1718 non-null   int64  
 6   결산월           1718 non-null   object 
 7   대표자명          1718 non-null   object 
 8   홈페이지          1572 non-null   object 
 9   지역            1718 non-null   object 
 10  사업자등록번호       690 non-null    object 
 11  주소            690 non-null    object 
 12  업종코드          690 non-null    float64
 13  설립일           690 non-null    float64
 14  최대주주_성명       658 non-null    object 
 15  최대주주_관계       658 non-null    object 
 16  최대주주_주식종류     658 non-null    object 
 17  최대주주_기초주식수    658 non-null    object 
 18  최대주주_기초주식지분율  658 non-null  

결측치 채우기

In [29]:
import pandas as pd

# 쉼표 제거 및 NaN 값 처리
숫자열 = ['소액주주수', '주주수', '소액주주_주식수', '총주식수']
company_info[숫자열] = company_info[숫자열].replace({',': '', '-': 0}, regex=True)


# 빈 문자열이면 0으로 대체
company_info[숫자열] = company_info[숫자열].fillna(0)

# 정수형으로 변환
company_info[숫자열] = company_info[숫자열].astype('int64')

company_info

,고유번호,종목코드,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,...,최대주주_기말주식지분율,소액주주수,주주수,소액주주비율,소액주주_주식수,총주식수,소액주주_주식지분율,감사의견,감사의견_강조사항,감사의견_핵심감사사항
0,1807738,475240,하나32호스팩,금융 지원 서비스업,기업인수합병,45378,12월,박종찬,NaN,서울특별시,...,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN
1,1592049,455900,엔젤로보틱스,특수 목적용 기계 제조업,"재활 전문 웨어러블 로봇, 근력 보조 무동력 웨어러블 슈트, 일상생활 보조 웨어러블...",45377,12월,공경철,http://www.angel-robotics.com/ko/,서울특별시,...,NaN,0,0,-,0,0,-,,,
2,1553129,420570,제이투케이바이오,기타 화학제품 제조업,화장품 원료(천연소재 바이오소재 등),45376,12월,이재섭,http://www.j2kbio.com,충청북도,...,28.49,9,21,42.85%,390000,5059840,7.7%,적정예외사항 없음(반기) 적정 적정,전기오류수정,
3,262266,437730,삼현,자동차 신품 부품 제조업,"스마트 액추에이터, 스마트 파워유닛",45372,12월,황성호,http://www.samhyun.co.kr,경상남도,...,NaN,10,18,55.56%,78764,8535856,0.92%,적정 적정 적정,,
4,260657,36220,오상헬스케어,의료용품 및 기타 의약 관련제품 제조업,"생화학 측정기 및 센서, 면역진단기기 및 마커, 분자진단기기 및 시약 등",45364,12월,홍승억,http://www.osanghc.com,경기도,...,NaN,3860,3866,99.84%,4064735,13084716,31.06%,적정 적정 적정,해당사항 없음 해당사항 없음 연결재무제표에 대한 주석 43에서 설명하고 있는 바와 ...,해당사항 없음 해당사항 없음 해당사항 없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713,165060,13030,하이록코리아,1차 철강 제조업,관이음쇠,32857,12월,"문휴건, 문창환 각자대표이사",http://www.hy-lok.com,부산광역시,...,20.54,7975,7995,99.75%,4823939,12856050,37.52%,적정 적정 적정,해당사항 없음 해당사항 없음 해당사항 없음,연결재무제표: 재고자산 평가충당금 추정의 적정성 평가\n별도재무제표 : 재고자산 평...
1714,156910,19550,SBI인베스트먼트,신탁업 및 집합투자업,벤처캐피탈,32757,12월,공동대표이사 안재광공동대표이사 소우 에이이치로,http://www.sbik.co.kr,서울특별시,...,NaN,34827,34832,99.99%,84456144,162066575,52.11%,적정 적정 적정,해당사항없음 해당사항 없음 해당사항 없음,수준3 금융상품공정가치 수준3 금융상품공정가치 수준3 금융상품공정가치
1715,110750,19570,리더스 기술투자,신탁업 및 집합투자업,금융투자,32581,12월,엄원진,http://www.geminivc.co.kr,서울특별시,...,25.64,13338,13344,99.99%,31491675,52470040,60.02%,적정 적정,,특수관계자 거래 공시사항 가. 핵심감사사항으로 결정한 이유\n재무제표에 대한 주석...
1716,136642,19590,엠벤처투자,신탁업 및 집합투자업,벤처캐피탈,32581,12월,심성보(SHIM SUNG BO),http://www.m-vc.co.kr,서울특별시,...,NaN,4281,4285,99.93%,82163147,113885542,72.10%,의견거절 적정 적정,1. 보통주식 및 투자전환사채 관련 공정가치평가 적정성\n 2. 비유동자산의 실재성...,별도 : 당기손익-공정가치측정금융자산의 공정가치 평가\n연결 : 당기손익-공정가치...


In [30]:
company_info[['소액주주비율', '소액주주_주식지분율']] = company_info[['소액주주비율', '소액주주_주식지분율']].replace('-', np.nan)

In [31]:
company_info

,고유번호,종목코드,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,...,최대주주_기말주식지분율,소액주주수,주주수,소액주주비율,소액주주_주식수,총주식수,소액주주_주식지분율,감사의견,감사의견_강조사항,감사의견_핵심감사사항
0,1807738,475240,하나32호스팩,금융 지원 서비스업,기업인수합병,45378,12월,박종찬,NaN,서울특별시,...,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN
1,1592049,455900,엔젤로보틱스,특수 목적용 기계 제조업,"재활 전문 웨어러블 로봇, 근력 보조 무동력 웨어러블 슈트, 일상생활 보조 웨어러블...",45377,12월,공경철,http://www.angel-robotics.com/ko/,서울특별시,...,NaN,0,0,NaN,0,0,NaN,,,
2,1553129,420570,제이투케이바이오,기타 화학제품 제조업,화장품 원료(천연소재 바이오소재 등),45376,12월,이재섭,http://www.j2kbio.com,충청북도,...,28.49,9,21,42.85%,390000,5059840,7.7%,적정예외사항 없음(반기) 적정 적정,전기오류수정,
3,262266,437730,삼현,자동차 신품 부품 제조업,"스마트 액추에이터, 스마트 파워유닛",45372,12월,황성호,http://www.samhyun.co.kr,경상남도,...,NaN,10,18,55.56%,78764,8535856,0.92%,적정 적정 적정,,
4,260657,36220,오상헬스케어,의료용품 및 기타 의약 관련제품 제조업,"생화학 측정기 및 센서, 면역진단기기 및 마커, 분자진단기기 및 시약 등",45364,12월,홍승억,http://www.osanghc.com,경기도,...,NaN,3860,3866,99.84%,4064735,13084716,31.06%,적정 적정 적정,해당사항 없음 해당사항 없음 연결재무제표에 대한 주석 43에서 설명하고 있는 바와 ...,해당사항 없음 해당사항 없음 해당사항 없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713,165060,13030,하이록코리아,1차 철강 제조업,관이음쇠,32857,12월,"문휴건, 문창환 각자대표이사",http://www.hy-lok.com,부산광역시,...,20.54,7975,7995,99.75%,4823939,12856050,37.52%,적정 적정 적정,해당사항 없음 해당사항 없음 해당사항 없음,연결재무제표: 재고자산 평가충당금 추정의 적정성 평가\n별도재무제표 : 재고자산 평...
1714,156910,19550,SBI인베스트먼트,신탁업 및 집합투자업,벤처캐피탈,32757,12월,공동대표이사 안재광공동대표이사 소우 에이이치로,http://www.sbik.co.kr,서울특별시,...,NaN,34827,34832,99.99%,84456144,162066575,52.11%,적정 적정 적정,해당사항없음 해당사항 없음 해당사항 없음,수준3 금융상품공정가치 수준3 금융상품공정가치 수준3 금융상품공정가치
1715,110750,19570,리더스 기술투자,신탁업 및 집합투자업,금융투자,32581,12월,엄원진,http://www.geminivc.co.kr,서울특별시,...,25.64,13338,13344,99.99%,31491675,52470040,60.02%,적정 적정,,특수관계자 거래 공시사항 가. 핵심감사사항으로 결정한 이유\n재무제표에 대한 주석...
1716,136642,19590,엠벤처투자,신탁업 및 집합투자업,벤처캐피탈,32581,12월,심성보(SHIM SUNG BO),http://www.m-vc.co.kr,서울특별시,...,NaN,4281,4285,99.93%,82163147,113885542,72.10%,의견거절 적정 적정,1. 보통주식 및 투자전환사채 관련 공정가치평가 적정성\n 2. 비유동자산의 실재성...,별도 : 당기손익-공정가치측정금융자산의 공정가치 평가\n연결 : 당기손익-공정가치...


In [32]:
# NA 값을 평균값으로 대체 (문자열을 제외하고 계산)
a = str(round(company_info['소액주주비율'].str.rstrip('%').astype(float).mean(), 2)) + '%'
b = str(round(company_info['소액주주_주식지분율'].str.rstrip('%').astype(float).mean(), 2)) + '%'

company_info['소액주주비율'] = company_info['소액주주비율'].fillna(a)
company_info['소액주주_주식지분율'] = company_info['소액주주_주식지분율'].fillna(b)

company_info

,고유번호,종목코드,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,...,최대주주_기말주식지분율,소액주주수,주주수,소액주주비율,소액주주_주식수,총주식수,소액주주_주식지분율,감사의견,감사의견_강조사항,감사의견_핵심감사사항
0,1807738,475240,하나32호스팩,금융 지원 서비스업,기업인수합병,45378,12월,박종찬,NaN,서울특별시,...,NaN,0,0,99.18%,0,0,52.05%,NaN,NaN,NaN
1,1592049,455900,엔젤로보틱스,특수 목적용 기계 제조업,"재활 전문 웨어러블 로봇, 근력 보조 무동력 웨어러블 슈트, 일상생활 보조 웨어러블...",45377,12월,공경철,http://www.angel-robotics.com/ko/,서울특별시,...,NaN,0,0,99.18%,0,0,52.05%,,,
2,1553129,420570,제이투케이바이오,기타 화학제품 제조업,화장품 원료(천연소재 바이오소재 등),45376,12월,이재섭,http://www.j2kbio.com,충청북도,...,28.49,9,21,42.85%,390000,5059840,7.7%,적정예외사항 없음(반기) 적정 적정,전기오류수정,
3,262266,437730,삼현,자동차 신품 부품 제조업,"스마트 액추에이터, 스마트 파워유닛",45372,12월,황성호,http://www.samhyun.co.kr,경상남도,...,NaN,10,18,55.56%,78764,8535856,0.92%,적정 적정 적정,,
4,260657,36220,오상헬스케어,의료용품 및 기타 의약 관련제품 제조업,"생화학 측정기 및 센서, 면역진단기기 및 마커, 분자진단기기 및 시약 등",45364,12월,홍승억,http://www.osanghc.com,경기도,...,NaN,3860,3866,99.84%,4064735,13084716,31.06%,적정 적정 적정,해당사항 없음 해당사항 없음 연결재무제표에 대한 주석 43에서 설명하고 있는 바와 ...,해당사항 없음 해당사항 없음 해당사항 없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713,165060,13030,하이록코리아,1차 철강 제조업,관이음쇠,32857,12월,"문휴건, 문창환 각자대표이사",http://www.hy-lok.com,부산광역시,...,20.54,7975,7995,99.75%,4823939,12856050,37.52%,적정 적정 적정,해당사항 없음 해당사항 없음 해당사항 없음,연결재무제표: 재고자산 평가충당금 추정의 적정성 평가\n별도재무제표 : 재고자산 평...
1714,156910,19550,SBI인베스트먼트,신탁업 및 집합투자업,벤처캐피탈,32757,12월,공동대표이사 안재광공동대표이사 소우 에이이치로,http://www.sbik.co.kr,서울특별시,...,NaN,34827,34832,99.99%,84456144,162066575,52.11%,적정 적정 적정,해당사항없음 해당사항 없음 해당사항 없음,수준3 금융상품공정가치 수준3 금융상품공정가치 수준3 금융상품공정가치
1715,110750,19570,리더스 기술투자,신탁업 및 집합투자업,금융투자,32581,12월,엄원진,http://www.geminivc.co.kr,서울특별시,...,25.64,13338,13344,99.99%,31491675,52470040,60.02%,적정 적정,,특수관계자 거래 공시사항 가. 핵심감사사항으로 결정한 이유\n재무제표에 대한 주석...
1716,136642,19590,엠벤처투자,신탁업 및 집합투자업,벤처캐피탈,32581,12월,심성보(SHIM SUNG BO),http://www.m-vc.co.kr,서울특별시,...,NaN,4281,4285,99.93%,82163147,113885542,72.10%,의견거절 적정 적정,1. 보통주식 및 투자전환사채 관련 공정가치평가 적정성\n 2. 비유동자산의 실재성...,별도 : 당기손익-공정가치측정금융자산의 공정가치 평가\n연결 : 당기손익-공정가치...


In [33]:
# 전처리를 마친 파일 저장
company_info.to_excel('다트전처리완료_2023.xlsx', index=False)
# company_info.to_excel('다트전처리완료_2022.xlsx', index=False)